# Main tester code

Last updated 06/24/2024

In [ ]:
# importing datetime module for now()
import datetime
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit_optimization.algorithms import MinimumEigenOptimizer

#2024 Imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.visualization import plot_histogram

from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime.fake_provider import FakeOsaka
from qiskit.providers.fake_provider import GenericBackendV2

from qiskit_aer import AerSimulator

from qiskit_algorithms.optimizers import SPSA

# Lots of import statements
import plotly.express as px
from qiskit import *
import matplotlib as mpl
import pandas as pd
from time import time
from copy import copy
from typing import List
from scipy.optimize import minimize

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)


from qiskit.circuit import Parameter, ParameterVector
from qiskit_optimization import QuadraticProgram
plt.rcParams['figure.dpi'] = 300

curdate = datetime.datetime.now()
print("Last update: ", curdate)

In [ ]:
import cain

accType = input(
    "Please select account type (type q for ibm_quantum and c for ibm_cloud, and r for RPI credentials)")

if accType == "q":
    chan = cain.qchan
    name = cain.qname
    token = cain.qtoken
    instance = cain.qinstance
    service = QiskitRuntimeService(channel=chan, token=token, instance=instance)
    
elif accType == "c":
    chan = cain.cchan
    name = cain.cname
    token = cain.ctoken
    instance = cain.cinstance
    service = QiskitRuntimeService(channel=chan, token=token, instance=instance)
    # Now takes user input to specify backend
    backendChoice = input(
        "Which backend would you like to use? (please type 'sim' or 'real')")

    if backendChoice == ("sim"):
        # service = QiskitRuntimeService()
        backend = service.backend("simulator_statevector")
        # backend = FakeOsaka()
        # backend = GenericBackendV2(num_qubits=24)
        
        # get a real backend from the runtime service
        # sim_backend = service.backend('ibm_rensselaer')
        # sim_backend = service.backend('ibm_osaka')

        # generate a simulator that mimics the real quantum system with the latest calibration results
        # backend_sim = AerSimulator.from_backend(sim_backend)
        
        config = 0
        locationChoice = input("Run locally or in the cloud? (0 for locally, 1 for cloud)")
        if locationChoice == 0:
            locconfig = 0
            backend = service.backend("simulator_statevector")
        elif locationChoice == 1:
            locconfig = 1
            # get a real backend from the runtime service
            # sim_backend = service.backend('ibm_rensselaer')
            sim_backend = service.backend('ibm_osaka')
            # generate a simulator that mimics the real quantum system with the latest calibration results
            backend = AerSimulator.from_backend(sim_backend)
        else:
            print("Please enter a valid choice (0 or 1)")
        
elif accType == "r":
    from qiskit_ibm_runtime import QiskitRuntimeService
 
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=cain.rtoken, overwrite=True)
    service = QiskitRuntimeService(channel="ibm_quantum") 
    # backend = service.backend("ibm_rensselaer")
    # target = backend.target

    locationChoice = input("Run locally or in the cloud? (0 for locally, 1 for cloud)")
    if locationChoice == 0:
        # service = QiskitRuntimeService()
        backend = service.backend("ibm_rensselaer")
        noise_model = NoiseModel.from_backend(backend)
        sim_noise = AerSimulator(noise_model=noise_model)
        passmanagerSim = generate_preset_pass_manager(optimization_level=2, backend=sim_noise)
        config = 2
        sim_backend = AerSimulator.from_backend(sim_noise)
    else:
        passmanager = generate_preset_pass_manager(optimization_level=2)
        config = 1
        backendChoice = "real"
        backend = service.backend("ibm_rensselaer")

    
else:
    print("Invalid input, please type 'q' or 'c' or 'r' ")

"""# Now takes user input to specify backend
backendChoice = input(
    "Which backend would you like to use? (please type 'sim' or 'real')")

if backendChoice == ("sim"):
    # service = QiskitRuntimeService()
    backend = service.backend("simulator_statevector")
    # backend = FakeOsaka()
    # backend = GenericBackendV2(num_qubits=24)
    
    # get a real backend from the runtime service
    # sim_backend = service.backend('ibm_rensselaer')
    # sim_backend = service.backend('ibm_osaka')

    # generate a simulator that mimics the real quantum system with the latest calibration results
    # backend_sim = AerSimulator.from_backend(sim_backend)
    
    config = 0
    locationChoice = input("Run locally or in the cloud? (0 for locally, 1 for cloud)")
    if locationChoice == 0:
        locconfig = 0
        backend = service.backend("simulator_statevector")
    elif locationChoice == 1:
        locconfig = 1
        # get a real backend from the runtime service
        # sim_backend = service.backend('ibm_rensselaer')
        sim_backend = service.backend('ibm_osaka')
        # generate a simulator that mimics the real quantum system with the latest calibration results
        backend = AerSimulator.from_backend(sim_backend)
    else:
        print("Please enter a valid choice (0 or 1)")
        
elif backendChoice == ("real"):
    config = 1
    realChoice = input(
        "Which real backend would you like to use? Enter the name of the machine (ex. 1 ='rensselaer', 2 = 'osaka')")
    if realChoice == "1":
        #backend = "ibm_osaka"
        backend = service.backend("ibm_rensselaer")
    else:
        print("please enter a valid real backend")
else:
    print("Please respond with a valid choice, print either sim or real with no extra characters")"""
 
service.backends()

# real_backend = service.backend(ibm_rensselaer)
# fake_rensselaer = AerSimulator.from_backend(real_backend)


## Attack set key (input the letter/number combonation ONLY)

| Set # | Nodes | Description | Graph |
| ---: | -----: | :---------------------------------- | ----------------------------- |
| B0 | 7 | A set used for benchmarking basic accuracy |
| B1 | 7 | A second set used for benchmarking basic accuracy |
| R1 | 12 | Reg set 2; Attack weights are 1.15 |
| R2 | 12 | Red set 2.2 Same set as 12 BUT attack weights are 5 |
| R3 | 14 | Reg Set 1 |
| L1 | 49 | Largest set that can run on a simulator (has 50qubits max) |
| L5 | 126 | The largest attack set. Cannot be run or checked classically |


In [ ]:
# Prepared Parameters
""" ## Parameters for Tests
==============================
- Data set: (1, 2, 3, 4, 5)
- Number of nodes: n
- Reps(Circuit Depth)
    - for QAOA: reps = repsQAOA
    - for VQE: reps = repsVQE
- Optimizer (for QAOA): _(ADAM | COBYLA | SPSA | SLSQP)_
- Ansatz (for VQE): _(EfficientSU2 | TwoLocal)_   """

print("Test is set to run on : " + str(backend))

print("Parameters for next test")
print("================")

# Prompt for set selection
attackSet = input("Which attack set would you like to run?")
if backendChoice == "sim":
    sTitle = " (Set: " + attackSet + ")"
    if attackSet == ("B0"):
        sset = 0
        n = 7
    elif attackSet == ("B1"):
        sset = 1
        n = 7
    elif attackSet == ("R1"):
        sset = 2
        n = 12
    elif attackSet == ("R2"):
        sset = 3
        n = 12
    elif attackSet == ("R3"):
        sset = 4
        n = 14
    elif attackSet == ("F1"):
        sset = 5
        n = 45
    elif attackSet == ("L1"):
        sset = 6
        n = 49
    elif attackSet == ("L5"):
        sset = 7
        n = 126
    elif attackSet == ("P1"):
        sset = 7
        n = 10
    elif attackSet == ("P2"):
        sset = 8
        n = 13
    elif attackSet == ("P3"):
        sset = 9
        n = 9
    elif attackSet == ("P4"):
        sset = 10
        n = 10
    elif attackSet == ("P5"):
        sset = 11
        n = 10
    elif attackSet == ("P6"):
        sset = 12
        n = 10
    elif attackSet == ("L2"):
        sset = 6
        n = 127
    elif attackSet == ("R2R"):
        sset = 13
        n = 12
    else:
        print("Please respond with a valid choice, print the number with no extra spaces (REMINDER: SIMULATORS CAN NOT RUN SETS WITH MORE THAN ~20 NODES)")

if backendChoice == "real":
    sTitle = " (Set: " + attackSet + ")"
    if attackSet == ("B0"):
        sset = 0
        n = 7
    elif attackSet == ("B1"):
        sset = 1
        n = 7
    elif attackSet == ("R1"):
        sset = 2
        n = 12
    elif attackSet == ("R2"):
        sset = 3
        n = 12
    elif attackSet == ("R3"):
        sset = 4
        n = 14
    elif attackSet == ("F1"):
        sset = 5
        n = 45
    elif attackSet == ("L1"):
        sset = 6
        n = 49
    elif attackSet == ("L5"):
        sset = 7
        n = 126
    elif attackSet == ("P1"):
        sset = 7
        n = 10
    elif attackSet == ("P2"):
        sset = 8
        n = 13
    elif attackSet == ("P3"):
        sset = 9
        n = 9
    elif attackSet == ("P4"):
        sset = 10
        n = 10
    elif attackSet == ("P5"):
        sset = 11
        n = 10
    elif attackSet == ("P6"):
        sset = 12
        n = 10
    elif attackSet == ("L2"):
        sset = 6
        n = 127
    elif attackSet == ("R2R"):
        sset = 13
        n = 12
    elif attackSet == ("L127"):
        sset = 14
        n = 127
    else:
        print("Please respond with a valid choice, print the number with no extra spaces")

print("Selected Set : " + attackSet)
print(n)

In [ ]:
from qiskit_ibm_runtime import Options


"""# Make a noise model
fake_backend = FakeOsaka()
noise_model = NoiseModel.from_backend(fake_backend)


# For Noisy Simulations
# Set options to include the noise model
options = Options()
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42
}

# Set number of shots, optimization_level and resilience_level
options.execution.shots = 10000
options.optimization_level = 0
options.resilience_level = 1
"""

# For Noiseless Simulations
# Set number of shots, optimization_level and resilience_level
options = Options()
options.execution.shots = 10000
options.optimization_level = 0
options.resilience_level = 1


In [ ]:
# SETTING UP ELISTS

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import datetime

# Setting a variable to the current time using now()
current_time = datetime.datetime.now()

G = nx.Graph()  # Graph G
G2 = nx.Graph()  # Graph G2
G3 = nx.Graph()  # Graph G3


G.add_nodes_from(np.arange(0, n, 1))
G2.add_nodes_from(np.arange(0, n, 1))
G3.add_nodes_from(np.arange(0, n, 1))


"""List is formatted as the following
    (x, y, z)
        x = # of the node (it's identifier)
        y = The node that x is connected to
        z = Weight of the line between x and y """


# Baseline Set B0
elist0 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 1),
    (0, 6, 1),
    (1, 0, 1),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 1),
    (1, 6, 1),
    (5, 6, 1)
]

# Baseline Set B1
elist1 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 1),
    (0, 6, 1),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 1),
    (1, 6, 1),
    (5, 6, 1)
]

# Attack Set R1
elist2 = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 3, 1.0),
    (0, 4, 1.15),
    (0, 5, 1.15),
    (0, 6, 1.15),
    (0, 7, 1.0),
    (0, 8, 1.15),
    (0, 9, 1.15),
    (0, 10, 1.15),
    (0, 11, 1.0),
    (1, 0, 1.15),
    (1, 4, 1.15),
    (1, 5, 1.15),
    (1, 6, 1.15),
    (1, 8, 1.15),
    (1, 9, 1.15),
    (1, 10, 1.15),
    (2, 7, 1.0),
    (3, 11, 1.0)
                    ]

# Attack Set R2
elist3 = [
    (0, 1, 1),
    (0, 2, 1),
    (0, 3, 1),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (0, 7, 1),
    (0, 8, 5),
    (0, 9, 5),
    (0, 10, 5),
    (0, 11, 1),
    (1, 4, 5),
    (1, 5, 5),
    (1, 6, 5),
    (1, 8, 5),
    (1, 9, 5),
    (1, 10, 5),
    (2, 7, 1),
    (3, 11, 1)]

# Attack Set R3
elist4 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 1),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (0, 7, 5),
    (0, 8, 5),
    (0, 9, 5),
    (0, 10, 5),
    (0, 11, 5),
    (0, 12, 5),
    (0, 13, 5),
    (1, 3, 1),
    (2, 4, 5),
    (2, 5, 5),
    (2, 6, 5),
    (2, 7, 5),
    (2, 8, 5),
    (2, 9, 5),
    (2, 10, 5),
    (2, 11, 5),
    (2, 12, 5),
    (2, 13, 5)]

# Attack Set F1 (45 nodes)
elist5 = [
    (0, 1, 1),
    (0, 2, 1),
    (0, 3, 1),
    (0, 4, 1),
    (0, 5, 1),
    (0, 6, 1),
    (0, 7, 1),
    (0, 8, 1),
    (0, 9, 1),
    (0, 10, 5),
    (0, 11, 5),
    (0, 12, 5),
    (0, 13, 5),
    (0, 14, 5),
    (0, 15, 5),
    (0, 16, 5),
    (0, 17, 5),
    (0, 18, 5),
    (0, 19, 5),
    (0, 20, 5),
    (0, 21, 5),
    (0, 22, 5),
    (0, 23, 5),
    (0, 24, 5),
    (0, 25, 5),
    (0, 26, 5),
    (0, 27, 5),
    (0, 28, 5),
    (0, 29, 5),
    (0, 30, 5),
    (0, 31, 5),
    (0, 32, 5),
    (0, 33, 5),
    (0, 34, 5),
    (0, 35, 5),
    (0, 36, 5),
    (0, 37, 5),
    (0, 38, 5),
    (0, 39, 5),
    (0, 40, 5),
    (0, 41, 5),
    (0, 42, 5),
    (0, 43, 5),
    (1, 2, 1),
    (1, 3, 1),
    (1, 4, 1),
    (1, 5, 1),
    (1, 6, 1),
    (1, 7, 1),
    (1, 8, 1),
    (1, 9, 1),
    (1, 10, 5),
    (1, 11, 5),
    (1, 12, 5),
    (1, 13, 5),
    (1, 14, 5),
    (1, 15, 5),
    (1, 16, 5),
    (1, 17, 5),
    (1, 18, 5),
    (1, 19, 5),
    (1, 20, 5),
    (1, 21, 5),
    (1, 22, 5),
    (1, 23, 5),
    (1, 24, 5),
    (1, 25, 5),
    (1, 26, 5),
    (1, 27, 5),
    (1, 28, 5),
    (1, 29, 5),
    (1, 30, 5),
    (1, 31, 5),
    (1, 32, 5),
    (1, 33, 5),
    (1, 34, 5),
    (1, 35, 5),
    (1, 36, 5),
    (1, 37, 5),
    (1, 38, 5),
    (1, 39, 5),
    (1, 40, 5),
    (1, 41, 5),
    (1, 42, 5),
    (1, 43, 5),
    (1, 44, 5),
    (2, 3, 1),
    (2, 5, 1),
    (3, 7, 1),
    (4, 5, 1),
    (7, 9, 1)]

# Set L2
elist6 = [
    (0, 1, 1),
    (0, 3, 1),
    (0, 5, 8.2),
    (0, 7, 7.6),
    (0, 8, 7.6),
    (0, 9, 9.7),
    (0, 9, 5.8),
    (0, 10, 9.3),
    (0, 11, 1),
    (0, 12, 8.1),
    (0, 13, 4.5),
    (0, 13, 1),
    (0, 16, 9.5),
    (0, 18, 7.6),
    (0, 21, 5.6),
    (0, 22, 6.2),
    (0, 23, 5.8),
    (0, 23, 5.8),
    (0, 26, 4.3),
    (0, 30, 4.0),
    (0, 31, 8.8),
    (0, 35, 6.7),
    (0, 35, 1),
    (0, 36, 1),
    (0, 39, 6.0),
    (0, 44, 1),
    (0, 45, 3.1),
    (0, 47, 8.9),
    (0, 49, 1),
    (0, 50, 4.9),
    (0, 51, 5.7),
    (0, 53, 1),
    (0, 55, 1),
    (0, 58, 8.7),
    (0, 66, 1),
    (0, 66, 9.4),
    (0, 67, 6.5),
    (0, 71, 9.7),
    (0, 73, 4.4),
    (0, 76, 1),
    (0, 76, 6.3),
    (0, 77, 4.5),
    (0, 77, 6.0),
    (0, 78, 5.3),
    (0, 79, 7.4),
    (0, 81, 1),
    (0, 83, 1),
    (0, 87, 1),
    (0, 87, 4.9),
    (0, 88, 8.2),
    (0, 88, 1),
    (0, 89, 5.3),
    (0, 89, 8.0),
    (0, 91, 6.0),
    (0, 92, 7.4),
    (0, 93, 1),
    (0, 93, 5.2),
    (0, 94, 9.5),
    (0, 97, 1),
    (0, 97, 6.9),
    (0, 97, 5.2),
    (0, 103, 7.8),
    (0, 110, 4.3),
    (0, 112, 3.8),
    (0, 114, 1),
    (0, 115, 1),
    (0, 118, 9.6),
    (0, 122, 1),
    (0, 125, 3.4),
    (0, 126, 3.7),
    (0, 127, 8.3),
    (1, 2, 9.7),
    (1, 7, 1),
    (1, 11, 6.6),
    (1, 12, 9.0),
    (1, 12, 6.9),
    (1, 15, 3.6),
    (1, 17, 4.7),
    (1, 18, 1),
    (1, 19, 6.0),
    (1, 20, 4.9),
    (1, 21, 4.3),
    (1, 21, 4.5),
    (1, 22, 5.6),
    (1, 22, 1),
    (1, 26, 3.7),
    (1, 28, 1),
    (1, 29, 7.6),
    (1, 29, 9.1),
    (1, 30, 6.1),
    (1, 32, 3.4),
    (1, 32, 1),
    (1, 33, 8.5),
    (1, 34, 1),
    (1, 35, 9.5),
    (1, 38, 6.0),
    (1, 47, 6.0),
    (1, 48, 4.5),
    (1, 52, 1),
    (1, 53, 4.9),
    (1, 58, 9.3),
    (1, 60, 1),
    (1, 61, 4.7),
    (1, 66, 1),
    (1, 69, 4.8),
    (1, 70, 3.9),
    (1, 72, 1),
    (1, 75, 4.3),
    (1, 79, 1),
    (1, 80, 1),
    (1, 81, 1),
    (1, 87, 6.4),
    (1, 97, 7.1),
    (1, 100, 7.5),
    (1, 104, 9.0),
    (1, 105, 3.3),
    (1, 105, 9.1),
    (1, 108, 1),
    (1, 109, 7.5),
    (1, 112, 8.1),
    (1, 114, 1),
    (1, 115, 9.4),
    (1, 120, 1),
    (1, 120, 7.5),
    (1, 121, 1),
    (1, 125, 4.5),
    (1, 126, 6.5)
]

# Elist P1 (Set 1 from Paper)
elist7 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (0, 7, 5),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 5),
    (1, 6, 5),
    (1, 7, 5),
    (8, 0, 1),
    (9, 1, 1)]


# Elist P2 (Set 2 from Paper)
elist8 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (0, 7, 5),
    (0, 8, 5),
    (0, 9, 5),
    (0, 10, 5),
    (0, 11, 5),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 5),
    (1, 6, 5),
    (1, 7, 5),
    (1, 8, 5),
    (1, 9, 5),
    (1, 10, 5),
    (1, 11, 5),
    (12, 0, 1)]


# Elist P3 (Set 3 from Paper)
elist9 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (5, 0, 1),
    (6, 0, 1),
    (7, 1, 1),
    (8, 1, 1)]


# Elist P4 (Set 4 from Paper)
elist10 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 5),
    (1, 6, 5),
    (7, 0, 1),
    (8, 0, 1),
    (9, 1, 1)]


# Elist P5 (Set 5 from Paper)
elist11 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (0, 7, 5),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 5),
    (1, 6, 5),
    (0, 7, 5),
    (8, 0, 1),
    (9, 1, 1)]


# Elist P6 (Set 6 from Paper)
elist12 = [
    (0, 1, 1),
    (0, 2, 5),
    (0, 3, 5),
    (0, 4, 5),
    (0, 5, 5),
    (0, 6, 5),
    (0, 7, 5),
    (1, 2, 5),
    (1, 3, 5),
    (1, 4, 5),
    (1, 5, 5),
    (1, 6, 5),
    (0, 7, 5),
    (8, 0, 1),
    (9, 1, 1)]

# Elist R2 (with random-esque weights)
elist13 = [
    (0, 1, 1),
    (0, 2, 1),
    (0, 3, 1),
    (0, 4, 7.4),
    (0, 5, 4.7),
    (0, 6, 5.4),
    (0, 7, 1),
    (0, 8, 5.7),
    (0, 9, 7.2),
    (0, 10, 5.4),
    (0, 11, 1),
    (1, 4, 4.7),
    (1, 5, 6.7),
    (1, 6, 6.9),
    (1, 8, 4.4),
    (1, 9, 3.4),
    (1, 10, 5.0),
    (2, 7, 1),
    (3, 11, 1)
]

In [ ]:
# SETTING UP INITIAL GRAPHS

setList = [elist0, elist1, elist2,
           elist3, elist4, elist5,
           elist6, elist7, elist8,
           elist9, elist10, elist11,
           elist12, elist13]

elist = setList[sset]

# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)
G2.add_weighted_edges_from(elist)
G3.add_weighted_edges_from(elist)
colors = ['r' for node in G.nodes()]
colors2 = ['r' for node in G2.nodes()]
colors3 = ['r' for node in G3.nodes()]

pos = nx.spring_layout(G)

# Condiser having the title as an argument of the draw_graph call
# This would allow any of the draw_graphs to be used and still have the correct title

def draw_graph(G, colors, pos, title):
    default_axes = plt.axes(frameon=True)
    # Titles Graph
    plot_title = title

    # Label variable for X-axis
    xax = ""

    # Label variable for Y-axis
    yax = ""

    # Calls to title
    plt.title(plot_title)
    plt.xlabel(xax)
    plt.ylabel(yax)

    # Main drawing call of the graph
    
    # style = nx.get_edge_attributes(G, 'style')
    nx.draw_networkx_edges(G, pos=pos)
    # nx.draw_networkx_edges(G, pos=pos, style=style)
    # nx.draw_networkx_edges(G, pos=pos, style='dashed')
    nx.draw_networkx(G, node_color=colors, node_size=600,
                     alpha=.8, ax=default_axes, pos=pos)
    # nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)

    # edge_color : color or array of colors (default='k')
    # Find a way to make cut lines into dotted lines

    # Create 2 edgelists. 1 with the non-cut edges, and another with only the cut edges

    # cut_edges = [(u, v) for u, v in G.edges if lut[u] != lut[v]]
    # uncut_edges = [(u, v) for u, v in G.edges if lut[u] == lut[v]]

    # nx.draw_networkx_edges(G, pos, edgelist=cut_edges,
    #                       style='dashdot', alpha=0.5, width=3)
    # nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=3)
    
def draw_graph2(G2, colors2, pos, title2):
    default_axes = plt.axes(frameon=True)
    # Titles Graph
    plot_title = title2

    # Label variable for X-axis
    xax = ""

    # Label variable for Y-axis
    yax = ""

    # Calls to title
    plt.title(plot_title)
    plt.xlabel(xax)
    plt.ylabel(yax)

    # Main drawing call of the graph
    
    # style = nx.get_edge_attributes(G, 'style')
    nx.draw_networkx_edges(G2, pos=pos)
    # nx.draw_networkx_edges(G, pos=pos, style=style)
    # nx.draw_networkx_edges(G, pos=pos, style='dashed')
    nx.draw_networkx(G2, node_color=colors2, node_size=600,
                     alpha=.8, ax=default_axes, pos=pos)
    # nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G2, 'weight')
    nx.draw_networkx_edge_labels(G2, pos=pos, edge_labels=edge_labels)
    
def draw_graph3(G3, colors3, pos, title3):
    default_axes = plt.axes(frameon=True)
    # Titles Graph
    plot_title = title3

    # Label variable for X-axis
    xax = ""

    # Label variable for Y-axis
    yax = ""

    # Calls to title
    plt.title(plot_title)
    plt.xlabel(xax)
    plt.ylabel(yax)

    # Main drawing call of the graph
    
    # style = nx.get_edge_attributes(G, 'style')
    nx.draw_networkx_edges(G3, pos=pos)
    # nx.draw_networkx_edges(G, pos=pos, style=style)
    # nx.draw_networkx_edges(G, pos=pos, style='dashed')
    nx.draw_networkx(G3, node_color=colors3, node_size=600,
                     alpha=.8, ax=default_axes, pos=pos)
    # nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G3, 'weight')
    nx.draw_networkx_edge_labels(G3, pos=pos, edge_labels=edge_labels)



### Brute Force Sim

In [ ]:
# BRUTE FORCE SIM

""" # Formats date for use in file saving
curTime = current_time.hour + ":" + current_time.minute + "_" + current_time.month + "/" + current_time.date + "/" + current_time.year
nx.draw(draw_graph(G, colors, pos))

# Saves graph as file in directory with timestamp
plt.savefig("BestBrute" + curTime ".png") """

# Computing the weight matrix from the random graph
w = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i, j, default=0)
        if temp != 0:
            w[i, j] = temp['weight']
print("Weight Matrix: ", w)


# n is the number of nodes
# If there are more than 20 nodes in an attack set, brute force will take the average computer a non-polynomial time to solve it. 
if n < 20: # This statement stops the program from attempting Brute Force if it won't be able to solve it
    best_cost_brute = 0
    for b in range(2**n):
        x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
        cost = 0
        for i in range(n):
            for j in range(n):
                cost = cost + w[i, j]*x[i]*(1-x[j])
        if best_cost_brute < cost:
            best_cost_brute = cost
            xbest_brute = x
        print('case = ' + str(x) + ' cost = ' + str(cost))

    print(xbest_brute)
    colors = ['r' if xbest_brute[i] == 0 else 'c' for i in range(n)]
    style = ['solid' if xbest_brute[i] == 0 else 'dashed' for i in range(n)]
    title = "Brute Force Solution"

    # Final draw of brute force graph
    draw_graph(G, colors, pos, title)
    print('\nBest solution = ' + str(xbest_brute) +
        ' cost = ' + str(best_cost_brute))
    print(style)
else:
    print("Brute Force Aborted, too many nodes! (Your set cannot be run through brute force)")

In [ ]:
# SETTING UP MAXCUT AND QUBO

# https://qiskit-community.github.io/qiskit-optimization/stubs/qiskit_optimization.applications.Maxcut.html

# Plotting functions
%config InlineBackend.figure_format = 'retina'

# Define our Qiskit Maxcut Instance
max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
# print("qubo: ", qp)
print(qp.prettyprint())

# Translate to Ising Hamiltonian
qubitOp, offset = qp.to_ising()

print("qubitOp: ", qubitOp)

In [ ]:
# Backend Error Mapping
from qiskit.visualization import plot_error_map

plot_error_map(backend, figsize=(30, 24))

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


# Pass manager master settings

generate_preset_pass_manager(
    optimization_level=3,
    backend=backend,
    target=None,
    basis_gates=None,
    inst_map=None,
    coupling_map=None,
    instruction_durations=None,
    backend_properties=None,
    timing_constraints=None,
    initial_layout=None,
    layout_method=None,
    routing_method=None,
    translation_method=None,
    scheduling_method='asap', # scheduling_method (str): The pass to use for scheduling instructions.
    approximation_degree=1.0,
    seed_transpiler= 42, # seed_transpiler (int): Sets random seed for the stochastic parts of the transpiler
    unitary_synthesis_method='default',
    unitary_synthesis_plugin_config=None,
    hls_config=None,
    init_method=None,
    optimization_method=None,
) 

In [ ]:
# Setting up noise simulator
"""from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)"""

service = QiskitRuntimeService()
backend = service.backend("ibm_rensselaer")
noise_model = NoiseModel.from_backend(backend)

sim_noise = AerSimulator(noise_model=noise_model)

passmanagerSim = generate_preset_pass_manager(optimization_level=3, backend=sim_noise)

## Cost Function Setup

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

QAOARuntimeArray = []


def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    # isa_circuits = passmanager.run(ansatz)
    isa_circuits = pm.run(ansatz)
    
    #pubs = (isa_circuits, hamiltonian, params)
    # cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    #cost = estimator.run([pubs]).result()[0].data.evs
    # print(cost.pub_result().values[0])
    
    #hamiltonian_isaQ = hamiltonian.apply_layout(isa_circuits.layout)
    hamiltonian_isaQ = hamiltonian.apply_layout(ansatz.layout)
    pubs = (isa_circuits, hamiltonian_isaQ, params)
    # result = estimator.run(pubs=[pub]).result()
    # cost = result[0].data.evs[0]
    cost = estimator.run([pubs]).result()[0].data.evs
    #print(cost.usage_estimation)
    QAOARuntimeArray.append(cost.usage_estimation)

    
    return cost

def get_cost_func(ansatz, hamiltonian, estimator):
    # isa_circuits = passmanager.run(ansatz)
    isa_circuits = pm.run(ansatz)
    def eval_energy(params):
        # hamiltonian_isaQ = hamiltonian.apply_layout(isa_circuits.layout)
        hamiltonian_isaQ = hamiltonian.apply_layout(ansatz.layout)
        pubs = (isa_circuits, hamiltonian_isaQ, params)
        energy = estimator.runestimator.run([pubs]).result()[0].data.evs
        QAOARuntimeArray.append(cost.usage_estimation)


        return energy

    return eval_energy


## Callback Setup for both Algorithms

As it stands now, we are unable to save intermediate results from the iteration process, view the value of the cost function per iteration, nor are we able to monitor the progress of the routine. Callback functions are a standard way for users to obtain additional information about the status of an iterative algorithm. The standard SciPy callback routine allows for returning only the interim vector at each iteration. However, it is possible to do much more than this. Here we show how to use a mutable object, such as a dictionary, to store the current vector at each iteration, for example in case we need to restart the routine due to failure, and also return the current iteration number and average time per iteration.

In [ ]:
import time
from time import perf_counter
from humanfriendly import format_timespan

def buildCallback(ansatz, hamiltonian, estimator, callback_dict, maxiter, testAttempts, passmanager):
    """Return callback function that uses Estimator instance,
    and stores intermediate values into a dictionary.

    Parameters:
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance
        callback_dict (dict): Mutable dict for storing values

    Returns:
        Callable: Callback function object
    """

    def callback(current_vector):
        """Callback function storing previous solution vector,
        computing the intermediate cost value, and displaying number
        of completed iterations and average time per iteration.

        Values are stored in pre-defined 'callback_dict' dictionary.

        Parameters:
            current_vector (ndarray): Current vector of parameters
                                      returned by optimizer
        """
        # Keep track of the number of iterations
        callback_dict["iters"] += 1
        # Set the prev_vector to the latest one
        callback_dict["prev_vector"] = current_vector
        # Compute the value of the cost function at the current vector
        """callback_dict["cost_history"].append(
            estimator.run(ansatz, hamiltonian,
                          parameter_values=current_vector).result().values[0]
        )"""
        # isa_circuits = passmanager.run(ansatz)
        isa_circuits = pm.run(ansatz)
        hamiltonian_isaQ = hamiltonian.apply_layout(isa_circuits.layout)
        pubs = (isa_circuits, hamiltonian_isaQ, current_vector)
        callback_dict["cost_history"].append(
            estimator.run([pubs]).result()[0].data.evs
        )
        # Grab the current time
        current_time = time.perf_counter()
        # Find the total time of the execute (after the 1st iteration)
        if callback_dict["iters"] > 1:
            callback_dict["_total_time"] += current_time - \
                callback_dict["_prev_time"]
        # Set the previous time to the current time
        callback_dict["_prev_time"] = current_time
        # Compute the average time per iteration and round it
        time_str = (
            round(callback_dict["_total_time"] /
                  (callback_dict["iters"] - 1), 2)
            if callback_dict["_total_time"]
            else "-"
        )

        if callback_dict["iters"] % maxiter != 0:
            curIter = callback_dict["iters"] % maxiter
        else:
            curIter = maxiter

        # Total number of iterations for all the specified tests
        totalIter = maxiter * testAttempts
        
        testNum = (callback_dict["iters"] // maxiter) + 1

        # Determines which test is currently in progress
        if testNum != testAttempts:
            curTest = testNum
        else:
            curTest = testAttempts

        # Finds total remaining iterations for all remaining tests
        remainingIters = totalIter - callback_dict["iters"]

        # Finds how many reps until a test is complete
        iterRem = maxiter - curIter
        
        testPercent = 100*(round(curIter/maxiter, 2))
        
        ovrPercent = 100*(round(callback_dict["iters"] / totalIter, 2))

        # Estimates remaining time based on average time per iteration
        if time_str != "-":
            total_testTime = round((float(time_str) * iterRem), 0)
            total_runTime = round(
                (float(time_str) * remainingIters), 0)
            testTimeRem = format_timespan(total_testTime)
            runtimeRem = format_timespan(total_runTime)
        else:
            testTimeRem = "-"
            runtimeRem = "-"
        # Print to screen on single line
        """print(
            "Test #: [{}] Iters. done: {} / {} [Avg. time per iter: {}] [Current Test Time Remaining: {}] [Total Est. Time Remaining {}]".format(
                curTest, curIter, maxiter, time_str, testTimeRem, runtimeRem),
            end="\r",
            flush=True,
        )
        
        # Print to screen on single line
        print(
            "Test #: [{}] Iters. done: {}/{} [Avg. time per iter: {}] [{}% Complete] [Total Est. Time Remaining {}]".format(
                curTest, curIter, maxiter, time_str, testPercent, runtimeRem),
            end="\r",
            flush=True,
        )"""
        
        # Print to screen on single line
        print(
            "Overall Test: [{}/{}] Iters. done: {}/{}  [Avg. time per iter: {}] [{}% Complete] [Total Est. Time Remaining {}]".format(
                curTest, testAttempts, curIter, maxiter, time_str, ovrPercent, runtimeRem),
            end="\r",
            flush=True,
        )

    return callback

In [ ]:
from qiskit.visualization import plot_distribution
from collections import Counter
    
# Output formatting etc.

def resultBreakdown(algo, samp_dist, resultArray, callback_dict, maxiter, testAttempts):
    """
    Main QAOA function, uses Callback to track cost history and other intermediate values

    Parameters:
        algo (string): Either QAOA or VQE, used in graph title
        samp_dist (array): Sample Distributions from max_cut result.
        resultArray (array): Results appended to the array.
        callback_dict (dictionary): has time and iteration information to return to the user.
        maxiter (int): Passed to set bounds of graph.

    Returns:
        xQAOA [List]: Binary String Results
    
    
    """

    plot_distribution(samp_dist, figsize=(15, 5)) # Plot distribution of sampled bitstrings
    
    freqListKey = list(Counter(resultArray).keys()) # equals to list(set(words))
    freqList = list(Counter(resultArray).values()) # counts the elements' frequency

    temp = 0
    print("\n")
    # max(c, key=c.get)
    print(freqList)
    print("\n")
    for k in freqListKey:
        print('Occurences of ', k ,': ', freqList[temp])
        temp += 1
        
    mostFreqResultIndex = freqList.index(max(freqList))
    mostFreqRes = resultArray[mostFreqResultIndex]
    
    cleaned_mostFreqRes = mostFreqRes.strip("[]").replace(" ", "")
    mFRList = [int(char) for char in cleaned_mostFreqRes]
    print(mFRList)


    # print("\nTotal Iterations: ", callback_dict_Q["iters"])
    avgTimePerTest = callback_dict["_total_time"] / testAttempts
    avgIters = callback_dict["iters"] / testAttempts
    print("\nTotal Time: ", callback_dict["_total_time"]) # Total runtime
    print("\nAverage Est. Time per test: ", str(round(avgTimePerTest, 2)),' seconds') # Time per test
    print("\n Test Parameters")
    print("\n =======================")
    print("\n Algorithm: QAOA")
    print("\n Data Set: ", str(attackSet))
    if config == 0:
        location = "Local"
    elif config == 1:
        location = "Cloud"
    elif config == 2:
        location = "Local/Noisy"
    print("\n Location processed: ", location) # Should be either local or cloud
    # print("\n Optimizer: ", optimizer) #COBYLA or SPSA

    # Graph cut of most frequently found solution
    colors2 = ['r' if mFRList[i] == 0 else 'c' for i in range(n)]
    title2 = algo + 'Most Frequent Solution' + sTitle
    style = ['solid' if mFRList[i] == 0 else 'dashed' for i in range(n)]
    print(colors2)
    print(mFRList)
    draw_graph2(G, colors2, pos, title2)

    fig, ax = plt.subplots()
    ax.plot(range(callback_dict["iters"]), callback_dict["cost_history"])

    xlim = maxiter*2
    ax.set_xlim([0, xlim])
    plot_title = algo + ' Cost Iteration (Attack Set: ' + str(attackSet) + ')'
    # plot_title = 'QAOA Cost Iteration (Opt: SPSA) (qReps: ' + str(qReps) + ') (Attack Set: ' + str(attackSet) + ')'

    plt.title(plot_title)
    ax.set_xlabel("Iterations")
    ax.set_ylabel("Cost")


#### Parameters for QAOA / VQE Shots and Circuit Depth

In [ ]:

# sets reps for QAOA function (Optimal Range: 1-2). 1 proved to be very efficient and accurate on set R2
qReps = 1

# sets reps for VQE ansatz (Optimal Range: 2-3). 3 isn't great at finding ideal solution, stick with 2 for now
vReps = 2

# Shots for QAOA sampler
qShots = 8192

# Shots for VQE sampler
vShots = 8192

# Max iterations for SPSA in QAOA (Range: 100-150)
# maxiterQAOA = 150

# Max iterations for COBYLA in QAOA (Range: 25-35)
maxiterQAOA = 30

# Max iterations for L-BGFS-B (still testing for optimal)
# maxiterQAOA = 10

# Max iterations for SPSA in VQE (Range: 250-350)
# maxiterVQE = 300

# Max iterations for COBYLA in VQE (Range: 250-350)
maxiterVQE = 300

# QAOA

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
# from qiskit_aer import AerSimulator
 
# get a real backend from the runtime service
# service = QiskitRuntimeService(channel=chan, token=token, instance=instance)
# backend = FakeOsaka()



#fakeBackend = service.backend('ibm_osaka')
 
# generate a simulator that mimics the real quantum system with the latest calibration results
#backend_sim = AerSimulator.from_backend(fakeBackend)

In [ ]:
from qiskit.circuit.library import QAOAAnsatz

qAnsatz = QAOAAnsatz(qubitOp, reps=qReps)

circChoice = input("Would you like to print the circuit? ( y / n )")
if circChoice == "y":
    qAnsatz.decompose(reps=3).draw("mpl")
else:
    print("Circuit not printed")

#if n < 20:
    # Draw
    #qAnsatz.decompose(reps=3).draw("mpl")
#else:
    #print("Circuit can not be displayed, too many nodes!")
    
qAnsatz.decompose(reps=qReps).draw("mpl")

# User specifies how many times to run the test
testAttempsChoice = input("How many times should each algorithm be run?")
testAttempts = int(testAttempsChoice)

In [ ]:
#ansatz_isaQ = passmanager.run(qAnsatz)
ansatz_isaQ = pm.run(qAnsatz)

hamiltonian = qubitOp
hamiltonian_isaQ = qubitOp.apply_layout(ansatz_isaQ.layout)

ansatz_isaQ.decompose(reps=1).draw("mpl")
# hamiltonian_isaQ.decompose(reps=3).draw("mpl")

In [ ]:
QAOAResultArray = []

QuantumExecution = []

''' qRes is an OptimizerResult in the following format:
            Returns
                fun (float): The final value of the minimization.
                jac (POINT): The final gradient of the minimization.
                nfev (int): The total number of function evaluations.
                nit (int): The total number of iterations.
                njev (int): The total number of gradient evaluations.
                x (POINT): The final point of the minimization.
                '''

callback_dict_Q = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
    "_total_time": 0,
    "_prev_time": None,
}

In [ ]:
def maxcutQAOA(estimator, sampler, hamiltonian, ansatz, optimizer, testAttempts, maxiterQAOA, passmanager):
    """Main QAOA function, uses Callback to track cost history and other intermediate values

    Parameters:
        estimator (Estimator): Estimator primitive instance
        sampler (Sampler): Sampler primitive instance
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        optimizer (string): Optimizer that gets used by the scipy minimize function. Optimal is COBYLA for simulations and SPSA for anything with noise.
        testAttempts (int): set by input from the user; How many tests will be run with current parameters
        maxiterQAOA (int): Sets iterations within each test (25-35 for QAOA)

    Returns:
        xQAOA [List]: Binary String Results
        QAOAResultArray: Dictionary of results appended from xQAOA
        Graph 1: Most common output strings
        Graph 2: Cost/Iteration History for optimizing number of iterations per test
        Prints list of occurances output strings found in xQAOA
        Prints list of parameters"""   
        
    callbackQ = buildCallback(ansatz=ansatz, hamiltonian=hamiltonian, estimator=estimator, callback_dict=callback_dict_Q, maxiter=maxiterQAOA, testAttempts=testAttempts, passmanager=passmanager)
    testNum = 1  # Counts which test the algorithm is currently on
    
    """if optimizer == "SPSA":
        x0q = 2 * np.pi * np.random.rand(ansatz_isaQ.num_parameters)
        spsaMin = SPSA.minimize(self=SPSA, fun=cost_func(params=x0q, ansatz=qAnsatz, hamiltonian=qubitOp, estimator=estimator), x0=x0q)
        optimizer = spsaMin"""
    
    x0q = 2 * np.pi * np.random.rand(ansatz_isaQ.num_parameters)
    for i in range(testAttempts):
        # To begin the routine, we start by specifying a random initial set of parameters,
        # x0q = 2 * np.pi * np.random.rand(ansatz.num_parameters)
        # x0q = 2 * np.pi * np.random.rand(ansatz_isaQ.num_parameters)
        
        # SPSAMin = spsa.minimize(fun=cost_func(params=x0q, ansatz=qAnsatz, hamiltonian=qubitOp, estimator=estimator), x0=x0q)
        
        # Minimize (using scipy minimizer)
        Res = minimize(
            #cost_func(params, ansatz, hamiltonian, estimator, passmanager=passmanager),
            cost_func,
            x0=x0q,
            # args=(ansatz_isaQ, hamiltonian_isaQ, estimator),
            args=(ansatz, hamiltonian, estimator),
            method=optimizer,
            callback=callbackQ,
            options={'maxiter': maxiterQAOA}
        )

        qc = ansatz.assign_parameters(Res.x) # Assign solution parameters to ansatz
        qc.measure_all() # Add measurements to our circuit
        # qc_isa = passmanager.run(qc)
        qc_isa = passmanager.run(qc)
        #qc_isa.draw(output="mpl", idle_wires=False, style="iqp")

        result = sampler.run([qc_isa]).result() # get result from sampler
        samp_distQ = result[0].data.meas.get_counts() # sample the distribution of results
        xQAOA = max_cut.sample_most_likely(samp_distQ) # Use sample distribution in max_cut equation
        # xQAOA is a binary bitstring with the length of the number of nodes in the graph. (ex. 10010110110)

        # print(xQAOA)
        # print("Execution time: ", result.usage_estimation)
        
        QuantumExecution.append(result.usage_estimation)

        QAOAResultArray.append(str(xQAOA))
        
    
    resultBreakdown(algo = "QAOA", samp_dist = samp_distQ, resultArray = QAOAResultArray, callback_dict = callback_dict_Q, maxiter = maxiterQAOA, testAttempts=testAttempts)
    
    """from qiskit.visualization import plot_distribution
    from collections import Counter

    plot_distribution(samp_dist, figsize=(15, 5)) # Plot distribution of sampled bitstrings
    
    freqListQKey = list(Counter(QAOAResultArray).keys()) # equals to list(set(words))
    freqListQ = list(Counter(QAOAResultArray).values()) # counts the elements' frequency

    temp = 0
    print("\n")
    # max(c, key=c.get)
    # print(freqListQ)
    # print("\n")
    for k in freqListQKey:
        print('Occurences of ', k ,': ', freqListQ[temp])
        temp += 1
        
    mostFreqResultIndex = freqListQ.index(max(freqListQ))
    #mostFreqResult = freqListQKey[mostFreqResultIndex]
    mostFreqRes = QAOAResultArray[mostFreqResultIndex]
    # mostFreqResult = list(QAOAResultArray[mostFreqResultKey])
    # print("\n Most frequent result: ", mostFreqResult)
    # mFRList = list(mostFreqResult)
    
    cleaned_mostFreqRes = mostFreqRes.strip("[]").replace(" ", "")
    mFRList = [int(char) for char in cleaned_mostFreqRes]
    # print(mFRList)

    # print("\nTotal Iterations: ", callback_dict_Q["iters"])
    avgTimePerTest = callback_dict_Q["_total_time"] / testAttempts
    avgItersQ = callback_dict_Q["iters"] / testAttempts
    print("\nTotal Time: ", callback_dict_Q["_total_time"]) # Total runtime
    print("\nAverage Est. Time per test: ", str(round(avgTimePerTest, 2)),' seconds') # Time per test
    print("\nTest Parameters")
    print("=======================")
    print("Algorithm: QAOA")
    print("Data Set: ", str(attackSet))
    if config == 0:
        location = "Local"
    elif config == 1:
        location = "Cloud"
    print("\nLocation processed: ", location) # Should be either local or cloud
    print("Optimizer: ", optimizer) #COBYLA or SPSA

    # Graph cut of most frequently found solution
    baseColor = mFRList[0]
    colors2 = ['c' if mFRList[i] == baseColor else 'r' for i in range(n)]
    title2 = 'QAOA Most Frequent Solution' + sTitle
    style = ['solid' if mFRList[i] == 0 else 'dashed' for i in range(n)]
    print(colors2)
    print(mFRList)
    draw_graph2(G, colors2, pos, title2)

    fig, ax = plt.subplots()
    ax.plot(range(callback_dict_Q["iters"]), callback_dict_Q["cost_history"])

    qxlim = maxiterQAOA*2
    ax.set_xlim([0, qxlim])
    plot_title = 'QAOA Cost Iteration (Attack Set: ' + str(attackSet) + ')'
    # plot_title = 'QAOA Cost Iteration (Opt: SPSA) (qReps: ' + str(qReps) + ') (Attack Set: ' + str(attackSet) + ')'

    plt.title(plot_title)
    ax.set_xlabel("Iterations")
    ax.set_ylabel("Cost")"""

In [ ]:
from qiskit_ibm_runtime import Session
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit_ibm_runtime import SamplerV2, EstimatorV2

print(backend)

print("Config: ", config)

if config == 0:
    estimator = StatevectorEstimator()
    sampler = StatevectorSampler()
    
    #sampler = Sampler(backend_options={"method": "statevector"})
    #estimator = Estimator(backend_options={"method": "statevector"})

    sampler2 = SamplerV2(backend)
    estimator2 = EstimatorV2(backend)
    
    
    
    optimizer = "COBYLA"
    # optimizer = "L-BFGS-B"
    # optimizer = "trust-krylov"
    # optimizer = "trust-constr"
    # optimizer = "SLSQP"
    
    pm = generate_preset_pass_manager(target=target, optimization_level=3)
    
    # maxcutQAOA(estimator = estimator, sampler = sampler, hamiltonian = qubitOp, ansatz = qAnsatz, optimizer = optimizer, testAttempts = testAttempts, maxiterQAOA = maxiterQAOA)
    maxcutQAOA(estimator = estimator, sampler = sampler, hamiltonian = hamiltonian_isaQ, ansatz = ansatz_isaQ, optimizer = optimizer, testAttempts = testAttempts, maxiterQAOA = maxiterQAOA, passmanager = pm)



elif config == 1:
    with Session(service=service, backend=backend) as session:
        # Configure estimator
        estimator = EstimatorV2(session=session, options={"resilience_level": 2})
        # estimator.options.default_shots = 10_000
        #estimator.options.dynamical_decoupling.enable = True
        #estimator.options.resilience_level = 1
        # estimator.options.resilience.zne_mitigation = True
        # estimator.options.resilience.zne_mitigation = True

        
        # Configure sampler
        sampler = SamplerV2(session=session, options={"default_shots": 2048})
        #sampler.options.default_shots = 10_000
        # sampler.options.seed = 1234
        #sampler.options.dynamical_decoupling.enable = True
        
        
        # spsa = SPSA(maxiter=maxiterQAOA)
        
        # spsaMin = spsa.minimize()
        
        # SPSAMin = SPSA.minimize(fun=cost_func(params=x0q, ansatz=qAnsatz, hamiltonian=qubitOp, estimator=estimator), x0=x0q)
        #optimizer = spsa
        # optimizer = "SPSA"
        optimizer = "COBYLA"
        
        passmanager = generate_preset_pass_manager(target=target, optimization_level=3)
        
        maxcutQAOA(estimator = estimator, sampler = sampler, hamiltonian = qubitOp, ansatz = qAnsatz, optimizer = optimizer, testAttempts = testAttempts, maxiterQAOA = maxiterQAOA, passmanager = passmanager)
        # maxcutQAOA(estimator = estimator, sampler = sampler, hamiltonian = hamiltonian_isaQ, ansatz = ansatz_isaQ, optimizer = optimizer, testAttempts = testAttempts, maxiterQAOA = maxiterQAOA)
        
        # Close the session since we are now done with it
        session.close()
        
if config == 2:
    sampler = SamplerV2(backend = sim_backend)
    estimator = EstimatorV2(backend = sim_backend)
    
    optimizer = "COBYLA"
    # optimizer = "L-BFGS-B"
    # optimizer = "trust-krylov"
    # optimizer = "trust-constr"
    # optimizer = "SLSQP"
    
    passmanager = passmanagerSim
    
    # maxcutQAOA(estimator = estimator, sampler = sampler, hamiltonian = qubitOp, ansatz = qAnsatz, optimizer = optimizer, testAttempts = testAttempts, maxiterQAOA = maxiterQAOA)
    maxcutQAOA(estimator = estimator, sampler = sampler, hamiltonian = hamiltonian_isaQ, ansatz = ansatz_isaQ, optimizer = optimizer, testAttempts = testAttempts, maxiterQAOA = maxiterQAOA, passmanager = passmanager)
    
print("Cost function execution times: " + QAOARuntimeArray)
print("Main execution times: " + QuantumExecution)

In [ ]:
# Code for running tests in BATCH

# To find the usage time for your session or batch, in qiskit-ibm-runtime 0.23 or later, run session.details()["usage_time"] or batch.details()["usage_time"].

# # Submit the circuit to the sampler
#job = sampler.run([isa_circuit])
 
#print(job.usage_estimation)

In [ ]:
"""
    Let's try optimizing the costs in all tests past 1,
        by taking the optimal point from the first result
        and using that as the initial point we can find a
        more accurate solution faster
        
        For QAOA
        
"""

<h1> VQE Runtime Session

In [ ]:
vAnsatz = TwoLocal(qubitOp.num_qubits, "ry", "cz",
             reps=vReps, entanglement="linear")

hamiltonian = qubitOp

num_params = vAnsatz.num_parameters


In [ ]:
def cost_func_vqe(params, circuit, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    pub = (circuit, hamiltonian, params)
    cost = estimator.run([pub]).result()[0].data.evs

#    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver, SamplingVQE
from qiskit_algorithms.optimizers import SPSA

# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(qubitOp)

x = max_cut.sample_most_likely(result.eigenstate)
print("energy:", result.eigenvalue.real)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))

In [ ]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, P_BFGS, SPSA, QNSPSA, BOBYQA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Estimator, BaseEstimatorV2
from qiskit_ibm_runtime import SamplerV2, EstimatorV2
from qiskit_ibm_runtime import Session

# we will iterate over these different optimizers
optimizers = [COBYLA(), L_BFGS_B(), SLSQP(), P_BFGS(), SPSA(maxiter=800) #, BOBYQA()
              ]
converge_counts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

with Session(service=service, backend=backend) as session:
    for i, optimizer in enumerate(optimizers):
        print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
        # algorithm_globals.random_seed = 50

        counts = []
        values = []
        estimatorV = Estimator(#backend=backend
                               # , options={"resilience_level": 2}
                               )
        # estimatorV = EstimatorV2(session=session, options={"resilience_level": 2})
        # estimatorV.options.dynamical_decoupling.enable = True

        def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)

        vqe = VQE(estimatorV, vAnsatz, optimizer, callback=store_intermediate_result)
        result = vqe.compute_minimum_eigenvalue(operator=qubitOp)
        converge_counts[i] = np.asarray(counts)
        converge_vals[i] = np.asarray(values)
        
    session.close()

print("\rOptimization complete      ");



import pylab

pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(converge_counts[i], converge_vals[i], label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers")
pylab.legend(loc="upper right");

In [ ]:
print(result)
cost_function_evals = result.cost_function_evals

In [ ]:
initial_pt1 = result.optimal_point

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(
        converge_counts[i],
        abs(ref_value - converge_vals[i]),
        label=type(optimizer).__name__,
    )
pylab.xlabel("Eval count")
pylab.ylabel("Energy difference from solution reference value")
pylab.title("Energy convergence for various optimizers")
pylab.yscale("log")
pylab.legend(loc="upper right");

In [ ]:
from qiskit.primitives import Sampler


# construct SamplingVQE
optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
samplerTemp =Sampler()
vqe = SamplingVQE(sampler=samplerTemp, ansatz=ry, optimizer=SPSA(maxiter=300))

# run SamplingVQE
result = vqe.compute_minimum_eigenvalue(qubitOp)

# print results
x = max_cut.sample_most_likely(result.eigenstate)
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))

In [ ]:
# create minimum eigen optimizer based on SamplingVQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
print(result.prettyprint())

In [ ]:
print(initial_pt1)

In [ ]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, P_BFGS, SPSA, QNSPSA, BOBYQA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Estimator, BaseEstimatorV2
from qiskit_ibm_runtime import SamplerV2, EstimatorV2
from qiskit_ibm_runtime import Session

initial_pt = initial_pt1
# we will iterate over these different optimizers
optimizers = [COBYLA(), L_BFGS_B(), SLSQP(), P_BFGS(), SPSA(maxiter=300) #, BOBYQA()
              ]
converge_counts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

with Session(service=service, backend=backend) as session:
    for i, optimizer in enumerate(optimizers):
        print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
        # algorithm_globals.random_seed = 50

        counts = []
        values = []
        estimatorV = Estimator(#backend=backend
                               # , options={"resilience_level": 2}
                               )
        # estimatorV = EstimatorV2(session=session, options={"resilience_level": 2})
        # estimatorV.options.dynamical_decoupling.enable = True

        def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)

        vqe2 = VQE(estimatorV, vAnsatz, optimizer, callback=store_intermediate_result, initial_point=initial_pt)
        result2 = vqe2.compute_minimum_eigenvalue(operator=qubitOp)
        converge_counts[i] = np.asarray(counts)
        converge_vals[i] = np.asarray(values)
        
    session.close()

print("\rOptimization complete      ");



import pylab

pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(converge_counts[i], converge_vals[i], label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers w/ Initial Point")
pylab.legend(loc="upper right");

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result3 = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = result3.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(
        converge_counts[i],
        abs(ref_value - converge_vals[i]),
        label=type(optimizer).__name__,
    )
    

pylab.xlabel("Eval count")
pylab.ylabel("Energy difference from solution reference value")
pylab.title("Energy convergence for various optimizers w/ Initial Point")
pylab.yscale("log")
pylab.legend(loc="upper right");

In [ ]:
cost_function_evals1 = result2.cost_function_evals

In [ ]:
print(
    f"cost_function_evals is {cost_function_evals1} with initial point versus {cost_function_evals} without it."
)

In [ ]:
colors3 = ["r" if result.x[i] == 0 else "c" for i in range(n)]
titleV = "VQE Graph"
draw_graph3(G3, colors3, pos, titleV)

In [ ]:
from qiskit_algorithms import SamplingVQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SPSA

# construct SamplingVQE
optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)

# run SamplingVQE
result = vqe.compute_minimum_eigenvalue(qubitOp)

# print results
x = max_cut.sample_most_likely(result.eigenstate)

print(x)

## VQE Method 3

In [ ]:
from qiskit.circuit.library import TwoLocal

vAnsatz = TwoLocal(qubitOp.num_qubits, "ry", "cz",
              reps=vReps, entanglement="linear")

hamiltonian = qubitOp

num_params = vAnsatz.num_parameters

vcircChoice = input("Would you like to print the circuit? ( y / n )")
if vcircChoice == "y":
    vAnsatz.decompose(reps=3).draw("mpl")
else:
    print("Circuit not printed")

#if n < 20:
    # Draw
    #qAnsatz.decompose(reps=3).draw("mpl")
#else:
    #print("Circuit can not be displayed, too many nodes!")
    
    
vAnsatz.decompose(reps=qReps).draw("mpl")

# User specifies how many times to run the test
testAttempsChoice = input("How many times should each algorithm be run?")
testAttempts = int(testAttempsChoice)
VQEResultArray = []


In [ ]:

''' vRes is an OptimizerResult in the following format:
            Returns
                fun (float): The final value of the minimization.
                jac (POINT): The final gradient of the minimization.
                nfev (int): The total number of function evaluations.
                nit (int): The total number of iterations.
                njev (int): The total number of gradient evaluations.
                x (POINT): The final point of the minimization.
                '''

callback_dict_V = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
    "_total_time": 0,
    "_prev_time": None,
}

In [ ]:
def maxcutVQE(estimator, sampler, hamiltonian, ansatz, optimizer, testAttempts, maxiterVQE):
    """Main QAOA function, uses Callback to track cost history and other intermediate values

    Parameters:
        estimator (Estimator): Estimator primitive instance
        sampler (Sampler): Sampler primitive instance
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        optimizer (string): Optimizer that gets used by the scipy minimize function. Optimal is COBYLA for simulations and SPSA for anything with noise.
        testAttempts (int): set by input from the user; How many tests will be run with current parameters
        maxiterQAOA (int): Sets iterations within each test (25-35 for QAOA)

    Returns:
        xQAOA [List]: Binary String Results
        QAOAResultArray: Dictionary of results appended from xVQE
        Graph 1: Most common output strings
        Graph 2: Cost/Iteration History for optimizing number of iterations per test
        Prints list of occurances output strings found in xVQE
        Prints list of parameters"""   
        
    callbackV = buildCallback(ansatz=vAnsatz, hamiltonian=hamiltonian, estimator=estimator, callback_dict=callback_dict_V, maxiter=maxiterVQE, testAttempts=testAttempts)
    testNum = 1  # Counts which test the algorithm is currently on
    x0v = 2 * np.pi * np.random.rand(vAnsatz.num_parameters)
    for i in range(testAttempts):
        # To begin the routine, we start by specifying a random initial set of parameters,
        # x0v = 2 * np.pi * np.random.rand(vAnsatz.num_parameters)
        
        # SPSA
        """def loss(x):
            bound = ansatz.bind_parameters(x)
            return np.real((StateFn(observable, is_measurement=True) @ StateFn(bound)).eval())
        
        spsa = SPSA(maxiter=300)
        result = spsa.optimize(ansatz.num_parameters, loss, initial_point=initial_point)"""
        
        #spsa = SPSA(**optimizer_config, callback=callbackV)
        #result_obj=spsa.minimize(vqe_func,x0v)
        #loss=result_obj.fun
        #x= result_obj.x
        #nfev=result_obj.nfev
    
        # Minimize (using scipy minimizer)
        Res = minimize(
            cost_func,
            x0=x0v,
            args=(ansatz, hamiltonian, estimator),
            method=optimizer,
            callback=callbackV,
            options={'maxiter': maxiterVQE}
        )

        vqc = ansatz.assign_parameters(Res.x) # Assign solution parameters to ansatz
        vqc.measure_all() # Add measurements to our circuit
        vqc_isa = passmanager.run(vqc)
        #qc_isa.draw(output="mpl", idle_wires=False, style="iqp")

        result = sampler.run([vqc_isa]).result() # get result from sampler
        samp_distV = result[0].data.meas.get_counts() # sample the distribution of results
        xVQE = max_cut.sample_most_likely(samp_distV) # Use sample distribution in max_cut equation
        # xQAOA is a binary bitstring with the length of the number of nodes in the graph. (ex. 10010110110)

        VQEResultArray.append(str(xVQE))
    
    
    resultBreakdown(algo = "VQE", samp_dist = samp_distV, resultArray = VQEResultArray, callback_dict = callback_dict_V, maxiter = maxiterVQE)

    
    """from qiskit.visualization import plot_distribution
    from collections import Counter

    plot_distribution(samp_dist, figsize=(15, 5)) # Plot distribution of sampled bitstrings
    
    freqListVKey = list(Counter(VQEResultArray).keys()) # equals to list(set(words))
    freqListV = list(Counter(VQEResultArray).values()) # counts the elements' frequency

    temp = 0
    print("\n")
    # max(c, key=c.get)
    print(freqListV)
    print("\n")
    for k in freqListVKey:
        print('Occurences of ', k ,': ', freqListV[temp])
        temp += 1
        
    mostFreqResultIndex = freqListV.index(max(freqListV))
    #mostFreqResult = freqListQKey[mostFreqResultIndex]
    mostFreqResV = VQEResultArray[mostFreqResultIndex]
    # mostFreqResult = list(QAOAResultArray[mostFreqResultKey])
    # print("\n Most frequent result: ", mostFreqResult)
    # mFRList = list(mostFreqResult)
    
    cleaned_mostFreqResV = mostFreqResV.strip("[]").replace(" ", "")
    mFRList = [int(char) for char in cleaned_mostFreqResV]
    print(mFRList)


    # print("\nTotal Iterations: ", callback_dict_Q["iters"])
    avgTimePerTest = callback_dict_V["_total_time"] / testAttempts
    avgItersQ = callback_dict_V["iters"] / testAttempts
    print("\nTotal Time: ", callback_dict_V["_total_time"]) # Total runtime
    print("\nAverage Est. Time per test: ", str(round(avgTimePerTest, 2)),' seconds') # Time per test
    print("\n Test Parameters")
    print("\n =======================")
    print("\n Algorithm: VQE")
    print("\n Data Set: ", str(attackSet))
    if config == 0:
        location = "Local"
    elif config == 1:
        location = "Cloud"
    print("\n Location processed: ", location) # Should be either local or cloud
    print("\n Optimizer: ", optimizer) #COBYLA or SPSA

    # Graph cut of most frequently found solution
    colors2 = ['r' if mFRList[i] == 0 else 'c' for i in range(n)]
    title2 = 'VQE Most Frequent Solution' + sTitle
    style = ['solid' if mFRList[i] == 0 else 'dashed' for i in range(n)]
    print(colors2)
    print(mFRList)
    draw_graph2(G, colors2, pos, title2)

    fig, ax = plt.subplots()
    ax.plot(range(callback_dict_Q["iters"]), callback_dict_Q["cost_history"])

    qxlim = maxiterQAOA*2
    ax.set_xlim([0, qxlim])
    plot_title = 'QAOA Cost Iteration (Attack Set: ' + str(attackSet) + ')'
    # plot_title = 'QAOA Cost Iteration (Opt: SPSA) (qReps: ' + str(qReps) + ') (Attack Set: ' + str(attackSet) + ')'

    plt.title(plot_title)
    ax.set_xlabel("Iterations")
    ax.set_ylabel("Cost")"""

## VQE Method 2

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

ansatz = EfficientSU2(qubitOp.num_qubits)
ansatz.decompose().draw("mpl", style="iqp")


num_params = ansatz.num_parameters
num_params


# Optimization

"""
To reduce the total job execution time, Qiskit primitives only accept circuits (ansatz) and observables (Hamiltonian) that conform
    to the instructions and connectivity supported by the target system (referred to as instruction set architecture (ISA) circuits and observables).

ISA circuit:

Schedule a series of qiskit.transpiler passes to optimize the circuit for a selected backend and make it compatible with the backend's ISA.
    This can be easily done with a preset pass manager from qiskit.transpiler and its optimization_level parameter.

The lowest optimization level does the minimum needed to get the circuit running on the device;
    it maps the circuit qubits to the device qubits and adds swap gates to allow all two-qubit operations.
    The highest optimization level is much smarter and uses lots of tricks to reduce the overall gate count.
    Since multi-qubit gates have high error rates and qubits decohere over time, the shorter circuits should give better results."""
    
# target = backend.target
# pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isaV = passmanager.run(ansatz)
ansatz_isaV.draw(output="mpl", idle_wires=False, style="iqp")

# ISA Observable

"""
Transform the Hamiltonian to make it backend-compatible before running jobs with Runtime Estimator V2.
Perform the transformation by using the apply_layout method of SparsePauliOp object."""
hamiltonian_isaV = hamiltonian.apply_layout(layout=ansatz_isaV.layout)


#Cost Function
"""Like many classical optimization problems, the solution to a VQE problem can be formulated as minimization of a scalar cost function.
    By definition, VQE looks to find the ground state solution to a Hamiltonian by optimizing the ansatz circuit parameters to minimize the
    expectation value (energy) of the Hamiltonian. With the Qiskit Runtime Estimator directly taking a Hamiltonian and parameterized ansatz,
    and returning the necessary energy, the cost function for a VQE instance is quite simple."""
    
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

# Cost History Dictionary for the cost function
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

In [ ]:
def maxcutVQE2(estimator, sampler, hamiltonian, ansatz, optimizer, testAttempts, maxiterVQE):
    """Main QAOA function, uses Callback to track cost history and other intermediate values

    Parameters:
        estimator (Estimator): Estimator primitive instance
        sampler (Sampler): Sampler primitive instance
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        optimizer (string): Optimizer that gets used by the scipy minimize function. Optimal is COBYLA for simulations and SPSA for anything with noise.
        testAttempts (int): set by input from the user; How many tests will be run with current parameters
        maxiterQAOA (int): Sets iterations within each test (25-35 for QAOA)

    Returns:
        xQAOA [List]: Binary String Results
        QAOAResultArray: Dictionary of results appended from xVQE
        Graph 1: Most common output strings
        Graph 2: Cost/Iteration History for optimizing number of iterations per test
        Prints list of occurances output strings found in xVQE
        Prints list of parameters"""   
        
    callbackV = buildCallback(ansatz=ansatz, hamiltonian=hamiltonian, estimator=estimator, callback_dict=callback_dict_V, maxiter=maxiterVQE, testAttempts=testAttempts)
    testNum = 1  # Counts which test the algorithm is currently on
    
    for i in range(testAttempts):
        # To begin the routine, we start by specifying a random initial set of parameters,
        x0v = 2 * np.pi * np.random.random(num_params)
        with Session(backend=backend) as session:
            estimator = Estimator(session=session)
            estimator.options.default_shots = 10000

            Res = minimize(
            cost_func,
            x0=x0v,
            args=(ansatz, hamiltonian, estimator),
            method=optimizer,
            callback=callbackV,
            options={'maxiter': maxiterVQE}
        )
        
        vqc = ansatz.assign_parameters(Res.x) # Assign solution parameters to ansatz
        vqc.measure_all() # Add measurements to our circuit
        vqc_isa = passmanager.run(vqc)
        #qc_isa.draw(output="mpl", idle_wires=False, style="iqp")

        result = sampler.run([vqc_isa]).result() # get result from sampler
        samp_distV = result[0].data.meas.get_counts() # sample the distribution of results
        
        xVQE = max_cut.sample_most_likely(samp_distV) # Use sample distribution in max_cut equation
        # xVQE is a binary bitstring with the length of the number of nodes in the graph. (ex. 10010110110)

        VQEResultArray.append(str(xVQE))
        
    
    resultBreakdown(algo = "VQE", samp_dist = samp_distV, resultArray = VQEResultArray, callback_dict = callback_dict_V, maxiter = maxiterVQE)

    
    """from qiskit.visualization import plot_distribution
    from collections import Counter

    plot_distribution(samp_dist, figsize=(15, 5)) # Plot distribution of sampled bitstrings
    
    freqListVKey = list(Counter(VQEResultArray).keys()) # equals to list(set(words))
    freqListV = list(Counter(VQEResultArray).values()) # counts the elements' frequency

    temp = 0
    print("\n")
    # max(c, key=c.get)
    print(freqListV)
    print("\n")
    for k in freqListVKey:
        print('Occurences of ', k ,': ', freqListV[temp])
        temp += 1
        
    mostFreqResultIndex = freqListV.index(max(freqListV))
    mostFreqResV = VQEResultArray[mostFreqResultIndex]
    
    cleaned_mostFreqResV = mostFreqResV.strip("[]").replace(" ", "")
    mFRList = [int(char) for char in cleaned_mostFreqResV]
    print(mFRList)


    # print("\nTotal Iterations: ", callback_dict_Q["iters"])
    avgTimePerTest = callback_dict_Q["_total_time"] / testAttempts
    avgItersQ = callback_dict_Q["iters"] / testAttempts
    print("\nTotal Time: ", callback_dict_Q["_total_time"]) # Total runtime
    print("\nAverage Est. Time per test: ", str(round(avgTimePerTest, 2)),' seconds') # Time per test
    print("\n Test Parameters")
    print("\n =======================")
    print("\n Algorithm: QAOA")
    print("\n Data Set: ", str(attackSet))
    if config == 0:
        location = "Local"
    elif config == 1:
        location = "Cloud"
    print("\n Location processed: ", location) # Should be either local or cloud
    print("\n Optimizer: ", optimizer) #COBYLA or SPSA

    # Graph cut of most frequently found solution
    colors2 = ['r' if mFRList[i] == 0 else 'c' for i in range(n)]
    title2 = 'QAOA Most Frequent Solution' + sTitle
    style = ['solid' if mFRList[i] == 0 else 'dashed' for i in range(n)]
    print(colors2)
    print(mFRList)
    draw_graph2(G, colors2, pos, title2)

    fig, ax = plt.subplots()
    ax.plot(range(callback_dict_Q["iters"]), callback_dict_Q["cost_history"])

    qxlim = maxiterQAOA*2
    ax.set_xlim([0, qxlim])
    plot_title = 'QAOA Cost Iteration (Attack Set: ' + str(attackSet) + ')'
    # plot_title = 'QAOA Cost Iteration (Opt: SPSA) (qReps: ' + str(qReps) + ') (Attack Set: ' + str(attackSet) + ')'

    plt.title(plot_title)
    ax.set_xlabel("Iterations")
    ax.set_ylabel("Cost")"""

In [ ]:
from qiskit_ibm_runtime import Session
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit_ibm_runtime import EstimatorV2, SamplerV2


if config == 0:
    estimator = StatevectorEstimator()
    sampler = StatevectorSampler()
    optimizer = "COBYLA"
    
    maxcutVQE2(estimator = estimator, sampler = sampler, hamiltonian = hamiltonian, ansatz = vAnsatz, optimizer = optimizer, testAttempts = testAttempts, maxiterVQE = maxiterVQE)


elif config == 1:
    with Session(service=service, backend=backend) as session:
        # Configure estimator
        estimator = EstimatorV2(session=session, options={"resilience_level": 2})
        # estimator.options.default_shots = 10_000
        estimator.options.dynamical_decoupling.enable = True
        #estimator.options.resilience_level = 1
        
        # Configure sampler
        sampler = SamplerV2(session=session)
        #sampler.options.default_shots = 10_000
        # sampler.options.seed = 1234
        sampler.options.dynamical_decoupling.enable = True
        
        spsa = SPSA(maxiter=maxiterVQE)
        
        # spsaMin = spsa.minimize()
        
        # SPSAMin = SPSA.minimize(fun=cost_func(params=x0q, ansatz=qAnsatz, hamiltonian=qubitOp, estimator=estimator), x0=x0q)
        #optimizer = spsa
        # optimizer = "SPSA"
        optimizer = "COBYLA"
        
        maxcutVQE2(estimator = estimator, sampler = sampler, hamiltonian = hamiltonian_isaV, ansatz = ansatz_isaV, optimizer = optimizer, testAttempts = testAttempts, maxiterVQE = maxiterVQE)
        
        # Close the session since we are now done with it
        session.close()

# xQAOA = max_cut.sample_most_likely(samp_distQ)
# xQAOA = max_cut.sample_most_likely(quasi_dist)
#xQAOA = max_cut.sample_most_likely(samp_dist)

# avgItersQ = callback_dict_Q["iters"] / testAttempts

# # plot results
# colors2 = ['r' if xQAOA[i] == 0 else 'c' for i in range(n)]
# title2 = 'QAOA Solution' + sTitle
# style2 = ['solid' if xQAOA[i] == 0 else 'dashed' for i in range(n)]
# draw_graph2(G2, colors2, pos2, title2)

In [ ]:
VQEResultList = []

callback_dict_V = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
    "_total_time": 0,
    "_prev_time": None,
}

# Run SamplingVQE in a runtime session
with Session(service=service, backend=backend) as session:
    #estimator = Estimator(options={"default_shots": 8192})
    estimator = Estimator(backend, options={"resilience_level": 2, "default_shots": 8192})
    sampler = Sampler(backend, options={"default_shots": 8192})
    sampler.options.dynamical_decoupling.enable = True
    sampler.options.twirling.enable_gates = True
    # sampler = Sampler(options={"default_shots": 8192})
    
    callbackV = buildCallback(ansatz=vAnsatz, hamiltonian=qubitOp, estimator=estimator, callback_dict=callback_dict_V, maxiter=maxiterVQE, testAttempts=testAttempts)
    optimizer = SPSA(maxiter=maxiterVQE, callback=callbackV)
    
    testNumV = 1
    for i in range(testAttempts):
        # Selecting a random starting point
        x0v = 2 * np.pi * np.random.random(num_params)
        
        # Minimizer
        vRes = minimize(
            cost_func,
            x0=x0v,
            args=(vAnsatz, hamiltonian, estimator),
            method="COBYLA",
            callback=callbackV,
            options={'maxiter': maxiterVQE}
        )
        
        # Assign solution parameters to ansatz
        Vqc = vAnsatz.assign_parameters(vRes.x)
        # Add measurements to our circuit
        Vqc.measure_all()
        # Sample ansatz at optimal parameters
        samp_distV = sampler.run(Vqc, default_shots=int(1e4)).result().quasi_dists[0]

        xVQE = max_cut.sample_most_likely(samp_distV)
        VQEResultList.append(str(xVQE))

        # testNumV = testNumV + 1

    session.close()


# xVQE = max_cut.sample_most_likely(samp_distV)

#colors3 = ['r' if xVQE[i] == 0 else 'c' for i in range(n)]
# style3 = ['solid' if xVQE[i] == 0 else 'dashed' for i in range(n)]

#print('\nAll Solutions: ')

#for x in VQEResultList:
    #print(x)

#title3 = 'VQE Solution' + sTitle
#draw_graph3(G3, colors3, pos3, title3)

In [ ]:
from collections import Counter

avgItersVQE = callback_dict_V["iters"] / testAttempts

freqListVKey = list(Counter(VQEResultList).keys()
                    )  # equals to list(set(words))
# counts the elements' frequency
freqListV = list(Counter(VQEResultList).values())

avgTimePerTestV = callback_dict_V["_total_time"]/testAttempts


temp = 0
for k in freqListVKey:
  print('Occurences of ', k, ': ', freqListV[temp])
  temp += 1
  

#print("\nTotal Iterations: ", callback_dict["iters"])
#print('Average Iterations per test: ', avgItersVQE)

print("\nTotal Time: ", callback_dict_V["_total_time"])
print("\nAverage Est. Time per test: ", str(round(avgTimePerTestV, 2)), ' seconds')



In [ ]:

labels, counts = np.unique(VQEResultList, return_counts=True)

ticks = range(len(counts))
# plt.bar(ticks, counts, align='center')
# plt.xticks(ticks, labels)

# Figure Size
fig, ax = plt.subplots(figsize=(16, 9))

ax.barh(labels, counts)

plt.ylabel("Solution String")
plt.xlabel("Occurences")
# Add Plot Title
ax.set_title('Occurances of VQE Solution Strings',
              loc='left', )

# Remove axes splines
for s in ['top', 'bottom', 'left', 'right']:
    ax.spines[s].set_visible(False)

# Remove x, y Ticks
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

# Add padding between axes and labels
ax.xaxis.set_tick_params(pad=5)
ax.yaxis.set_tick_params(pad=10)

# Add x, y gridlines
ax.grid(color='grey', linestyle='-.', linewidth=0.5, alpha=0.4)

# Add Text watermark
# fig.text(0.9, 0.15, '@EvanSpillane', fontsize=12, color='grey', ha='right', va='bottom', alpha=0.7)

# Show top values
ax.invert_yaxis()

# Add annotation to bars
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize=10, fontweight='bold',
             color='grey')

plt.show()


In [ ]:
fig, ax = plt.subplots()
ax.plot(range(callback_dict_Q["iters"]), callback_dict_Q["cost_history"])
avgItersQ = callback_dict_Q["iters"] / testAttempts


qxlim = maxiterQAOA
ax.set_xlim([0, qxlim])
plot_title = 'QAOA Cost Iteration (Opt: SPSA) (qReps: ' + str(qReps) + ') (Attack Set: ' + str(attackSet) + ')'
plt.title(plot_title)
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")

In [ ]:

fig, ax = plt.subplots()
avgItersVQE = callback_dict_V["iters"] / testAttempts

ax.plot(range(callback_dict_V["iters"]), callback_dict_V["cost_history"])
vxlim = avgItersVQE*2
#vxlim = 400
ax.set_xlim([0, vxlim])
plot_title = 'VQE Cost Iteration  (Opt: SPSA) (vReps: ' + str(vReps) + ') (Attack Set: ' +  str(attackSet) + ')'
plt.title(plot_title)
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")

In [ ]:
import qiskit
import qiskit_ibm_runtime

#Code written by Evan Spillane, for the Marist-IBM Joint Study

print(qiskit.version.get_version_info())
print(qiskit_ibm_runtime.version.get_version_info())